In [1]:
#Importing Required Libraries

import pytesseract
from PIL import Image, ImageEnhance, ImageFile
import cv2
import numpy as np
import math
from scipy import ndimage
import os
import re
import glob
import datetime
import pandas as pd 

In [2]:
# Setting paths for PyTesseract and processing folders

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
rec_path = "C:/Users/mohan/Desktop/Project Image/20191217/Received/"
inp_path = "C:/Users/mohan/Desktop/Project Image/20191217/Input/"
prs_path = "C:/Users/mohan/Desktop/Project Image/20191217/Process/"
wrk_path = "C:/Users/mohan/Desktop/Project Image/20191217/Working/"
out_path = "C:/Users/mohan/Desktop/Project Image/20191217/Output/"
rst_path = "C:/Users/mohan/Desktop/Project Image/20191217/Result/"
tmp_path = "C:/Users/mohan/Desktop/Project Image/20191217/Templates/"

In [3]:
# Reading file and append list of file name

filename = []
directory = os.fsencode(inp_path)
for file in os.listdir(directory):
    filename.append (os.fsdecode(file))

In [4]:
# Processing all images, extracting text and storing the text output
print('Initializing the Process.....')
image_count = len(filename)
print ('The Input Folder has', image_count, ' Images to be processed')
print (datetime.datetime.utcnow())  

for file in filename:
    # Enhancing the image for OCR - contrast
    print(file)
    im = Image.open(inp_path + file)
    enhancer = ImageEnhance.Contrast(im)
    enhanced_im = enhancer.enhance(1.5)
    enhanced_im.save(wrk_path + "contrast.jpg")
    
    # Enhancing the image for OCR - Sharpness
    im = Image.open(wrk_path + "contrast.jpg")
    enhancer = ImageEnhance.Sharpness(im)
    enhanced_im = enhancer.enhance(1.0)
    enhanced_im.save(wrk_path + "shrp_contrast.jpg")
    
    # Deskewing the image (source https://gist.github.com/lnlonSA/d38f436552102a3fdf2a306f84ef1299)
    img_before = cv2.imread(wrk_path + "shrp_contrast.jpg")
    img_gray = cv2.cvtColor(img_before, cv2.COLOR_BGR2GRAY)
    img_edges = cv2.Canny(img_gray, 100, 100, apertureSize=3)
    lines = cv2.HoughLinesP(img_edges, 1, math.pi / 180.0, 100, minLineLength=100, maxLineGap=5)
    angles = []
    for x1, y1, x2, y2 in lines[0]:
        cv2.line(img_before, (x1, y1), (x2, y2), (255, 0, 0), 1)
        angle = math.degrees(math.atan2(y2 - y1, x2 - x1))
        angles.append(angle)
        median_angle = np.median(angles)
        img_rotated = ndimage.rotate(img_gray, median_angle)
        #print ("Angle is {}".format(median_angle))
        cv2.imwrite(wrk_path + 'segment_rotated.jpg', img_rotated)
    
    
    # Removal of lines and boxes
    image = cv2.imread(wrk_path + 'segment_rotated.jpg')
    result = image.copy()
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    
    # Remove horizontal lines
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20,1))
    remove_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
    cnts = cv2.findContours(remove_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(result, [c], -1, (255,255,255), 3)
    
    # Remove vertical lines
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,15))
    remove_vertical = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
    cnts = cv2.findContours(remove_vertical, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        cv2.drawContours(result, [c], -1, (255,255,255), 3)
    
    cv2.imwrite(wrk_path + 'matched5.jpg', result)
    
     # Take an input image and create segments
    #img = Image.open(wrk_path + 'matched5.jpg')
    #date1_img = img.crop((1000, 1, 1499, 140))
    #date1_img.save(wrk_path + file + "Date_image1.jpg")
    
    #img = cv2.imread(wrk_path + file + "Date_image1.jpg")
    #width = 700
    #height = 150
    #dim = (width, height)
    # resize image
    #resized = cv2.resize(img, dim, interpolation = cv2.INTER_LANCZOS4)
    #cv2.imwrite(wrk_path + file + "Date_image.jpg", resized)

    
    #Scan the image and extract text - all
    img1 = Image.open(wrk_path + 'matched5.jpg')
    text1 = pytesseract.image_to_string(img1, lang='eng')
    #print(text1)
    
    #Writing the extracted Text to the text file - all
    f = open(prs_path + file + '.txt','w')
    f.write(text1)
    f.close()

     #Scan the image and extract text - DATE
    #img2 = Image.open(wrk_path + file + "Date_image.jpg")
    #text2 = pytesseract.image_to_string(img2, lang='eng')
    #print(text1)
    
    #Writing the extracted Text to the text file - DATE
    #f = open(prs_path + file + 'Date1.txt','w')
    #f.write(text2)
    #f.close()
    
    #Extract Date Elements from Text - Date
    date = None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search(r'Date (.*$)', line,  re.IGNORECASE)
            # Did we find a match?
            if match:
                # Yes, process it
                date = match.group(1)
                #print('Date: '+ date)
    if date is None:
        date = 'Not Found'
        #print('Date: '+ date)
    
    #Extract Elements from Text - UtilityCode
    UtilityCode = None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search(r'yCode (.*$)', line,  re.IGNORECASE)
            # Did we find a match?
            if match:
                # Yes, process it
                UtilityCode = match.group(1)
                #print('Utility Code: '+UtilityCode)
                
    if UtilityCode is None:
        UtilityCode = 'Not Found'
        #print('Utility Code: '+UtilityCode)
    
    #Extract Elements from Text - Sponsor Bank code
    SBC = None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search('ode(.*) U', line,  re.IGNORECASE)
            # Did we find a match?
            if match:
                # Yes, process it
                SBC = match.group(1)
                #print('Sponsor Bank Code: ' + SBC)
    if SBC is None:
        SBC = 'Not Found'
        #print('Sponsor Bank Code: ' + SBC)
    
    
    #Extract Elements from Text - Bank name
    bankName = None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search('ith Bank(.*)', line,  re.IGNORECASE)
            # Did we find a match?
            if match:
                # Yes, process it
                bankName = match.group(1)
                #print('Bank Name: ' + bankName)
                
    if bankName is None:
        bankName = 'Not Found'
        #print('Bank Name: ' + bankName)
    
    #Extract Elements from Text - Bank Ac No
    bankNo = None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search(r'Number(.*$)', line,  re.IGNORECASE)
            # Did we find a match?
            if match:
                # Yes, process it
                bankNo = match.group(1)
                #print('Bank A/c No: ' +bankNo)
    if bankNo is None:
        bankNo = 'Not Found'
        #print('Bank A/c No: ' +bankNo)
    
    
    #Extract Elements from Text - IFSC
    IFSC =  None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search(r'SC (.*)or', line,  re.IGNORECASE)
            # Did we find a match?
            if match:
                # Yes, process it
                IFSC = match.group(1)
                #print('IFSC Code: ' +IFSC)
    
    if IFSC is None:
        IFSC = 'Not Found'
        #print('IFSC Code: ' +IFSC)
        
    
    #Extract Elements from Text - Amount
    amount = None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search(r'upees(.*)only', line,  re.IGNORECASE)
            # Did we find a match?
            if match:
                # Yes, process it
                amount = match.group(1)
                #print('Amount: ' + amount)
                
    if amount is None:
        amount = 'Not Found'
        #print('Amount: ' + amount)
    
    #Extract Elements from Text - Amount Digits
    amount1 = None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search(r'Only (.*$)', line,  re.IGNORECASE)
            # Did we find a match?
            if match:
                # Yes, process it
                amount1 = match.group(1)
                #print('Amount: ' + amount)
                
    if amount1 is None:
        amount1 = 'Not Found'
        #print('Amount: ' + amount)
                              
    
    #Extract Elements from Text - Reference 1
    Ref1 = None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search('ference1 (\S+)', line)
            # Did we find a match?
            if match:
                # Yes, process it
                Ref1 = match.group(1)
                #print('Reference 1: ' + Ref1)
    if Ref1 is None:
        Ref1 = 'Not Found'
        #print('Reference 1: ' + Ref1)
                              
    #Extract Elements from Text - Reference 2
    Ref2 = None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search('ference2 (\S+)', line)
            # Did we find a match?
            if match:
                # Yes, process it
                Ref2 = match.group(1)
                #print('Reference 2: ' + Ref2)
    if Ref2 is None:
        Ref2 = 'Not Found'
        #print('Reference 2: ' + Ref2)
                              
    #Extract Date Elements from Text - Mobile Number
    mobno = None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search(r'No.(.*$)', line)
            # Did we find a match?
            if match:
                # Yes, process it
                mobno = match.group(1)
                #print('Mobile Number: ' + mobno)
    if mobno is None:
        mobno = 'Not Found'
        #print('Mobile Number: ' + mobno)
                              
    #Extract Date Elements from Text - Mobile Number
    email = None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search(r'mail ID(.*$)', line, re.IGNORECASE)
            # Did we find a match?
            if match:
                # Yes, process it
                email = match.group(1)
                #print('email id: ' + email)
    if email is None:
        email = 'Not Found'
        #print('email id: ' + email)
                              
     #Extract Date Elements from Text - authorise party
    auth = None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search('authorise (.*)', line,  re.IGNORECASE)
            # Did we find a match?
            if match:
                # Yes, process it
                auth = match.group(1)
                #print('Authorised Party: ' + auth)
    
    if auth is None:
        auth = 'Not Found'
        #print('Authorised Party: ' + auth)
                              
    
    #Extract Date Elements from Text - From Date
    frm_date = None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search(r'From (.*$)', line,  re.IGNORECASE)
            # Did we find a match?
            if match:
                # Yes, process it
                frm_date = match.group(1) 
                #print('From Date: '+ frm_date)
    if frm_date is None:
        frm_date = 'Not Found'
        #print('From Date: '+ frm_date)
                              
    
    #Extract Date Elements from Text - To Date
    
    to_date = None
    # Open the file for reading
    with open(prs_path + file + '.txt') as fd:
        # Iterate over the lines
        for line in fd:
            # Capture one-or-more characters of non-whitespace after the initial match
            match = re.search(r'To (.*$)', line)
            # Did we find a match?
            if match:
                # Yes, process it
                to_date = match.group(1)
                #print('To Date: '+ to_date)
    if to_date is None:
        to_date = 'Not Found'
        #print('To Date: '+ to_date)
    
    
    file1 = os.path.splitext(file)[0]
    f = open(out_path + file1 + '.txt','w')
    f.write('Date1: '+ date + '\n' + 'Sponsor Bank Code: ' + SBC + '\n' +'Utility Code: ' + UtilityCode + '\n' + 'Authorised Party: ' + auth + '\n' + 'Bank Name: ' + bankName + '\n' + 'Bank A/c No: ' + bankNo +'\n' + 'IFSC Code: ' + IFSC + '\n' + 'Amount1: ' + amount + '\n' + 'Amount: ' + amount1 + '\n' +  'Reference 1: ' + Ref1 + '\n' + 'Reference 2: ' + Ref2 + '\n' + 'Mobile Number: ' + mobno + '\n' + 'email id: ' + email + '\n' + 'From Date: '+ frm_date + '\n' + 'To Date: '+ to_date) 
    f.close()
    
print (datetime.datetime.utcnow())   
print ('All ', image_count, ' Images have been processed')

Initializing the Process.....
The Input Folder has 52  Images to be processed
2020-01-19 18:30:14.043241
DMI0001479325.jpg
DMI0001521361.jpg
DMI0001521429.jpg
DMI0001522059.jpg
DMI0001522519.jpg
DMI0001555119.jpg
DMI0001555165.jpg
DMI0001556133.jpg
DMI0001556572.jpg
DMI0001556987.jpg
DMI0002891950.jpg
DMI0003058055.jpg
DMI0003161097.jpg
DMI0003178994.jpg
DMI0003827456.jpg
DMI0003995461.jpg
DMI0004031671.jpg
DMI0004036307.jpg
DMI0004089080.jpg
DMI0004089725.jpg
DMI0004094222.jpg
DMI0004101334.jpg
DMI0004103935.jpg
DMI0004104296.jpg
DMI0004116496.jpg
DMI0004128392.jpg
DMI0004132320.jpg
DMI0004137656.jpg
DMI0004139048.jpg
DMI0004139120.jpg
DMI0004141156.jpg
DMI0004162025.jpg
DMI0004166121.jpg
DMI0004170384.jpg
DMI0004174019.jpg
DMI0004174079.jpg
DMI0004181250.jpg
DMI0004182271.jpg
DMI0004183644.jpg
DMI0004186445.jpg
DMI0004188003.jpg
DMI0004188903.jpg
DMI0004204317.jpg
DMI0004243672.jpg
DMI0004511925.jpg
DMI0005181487.jpg
DMI0005495338.jpg
DMI0005499417.jpg
DMI0005608522.jpg
DMIMT20190326